In [ ]:
{'공격 타입':['물리', '마법']}
{'물리 공격 방식': ['배기', '찌르기', '타격']}
{'마법 공격 방식': ['배기', '찌르기', '타격', '폭발']}
{'관통' : ['방어력 관통': "%", '마법 방어력 관통': "%"]}
{'명중력': ''}
{'상태이상': ['출혈', '독', '화상', '기절', '빙결', '마비', '수면', '침묵', '혼란', '실명', '매혹', '공포', '둔화', '속박', '약화']}
{'출혈, 독, 화상, 빙결': {'데미지 계수', '지속 시간'}}
{'기절, 마비, 수면, 침묵, 혼란, 실명, 매혹, 공포, 속박': {'지속시간'}}
{'둔화, 약화': {'%', '지속시간'}}
{'데미지': {'Status', 'Coef', 'Count'}}



{
    "Attack_Type": ["Physical", "Magical"],
    "Physical_Attack_Method": ["Slash", "Stab", "Strike"],
    "Magical_Attack_Method": ["Slash", "Stab", "Strike", "Explosion"],
    "Penetration": {
        "Physical_Defense_Penetration": "%",
        "Magical_Defense_Penetration": "%"
    },
    "Accuracy": "%",
    "Status_Effect": ["Bleed", "Poison", "Burn", "Stun", "Freeze", "Paralysis", "Sleep", "Silence", "Confusion", "Blindness", "Charm", "Fear", "Slow", "Root", "Weaken"],
    "Bleed_Poison_Burn_Freeze": {
        "Status": "",
        "Coefficient": "",
        "Duration": ""
    },
    "Stun_Paralysis_Sleep_Silence_Confusion_Blindness_Charm_Fear_Root": {
        "Duration": ""
    },
    "Slow_Weaken": {
        "Effect_Percentage": "",
        "Duration": ""
    },
    "Damage": {
        "Status": "",
        "Coefficient": "",
        "Count": ""
    }
}

In [13]:
from Status import Status
import json
import Combat

with open('sample_combat.json', 'r') as f:
    data = json.load(f)
    
class Item():
    def __init__(self, item):
        self.name = item['Item Name']
        self.type = item['Item Type']
        self.description = item['Item Description']
        self.restriction = item['Use Restriction']
        self.effect = item['Effect']

class Inventory():
    def __init__(self, max_inv):
        self.inventory = list()
        self.max_inv = max_inv
    
    def add_item(self, item):
        if len(self.inventory) < self.max_inv:
            self.inventory.append(item)
        else:
            print('max_inv')
    
    def remove_item(self, item):
        self.inventory.remove(item)

    def add_max_inv(self, value):
        self.max_inv = self.max_inv + value if self.max_inv + value > 0 else 0 

    def isFull(self):
        return len(self.inventory) < self.max_inv


class Equipment():
    def __init__(self):
        self.helmet = None
        self.armor = None
        self.pants = None
        self.gloves = None
        self.shoes = None
        self.ring1 = None
        self.ring2 = None
        self.earring1 = None
        self.earring2 = None
        self.necklace = None
        self.right_hand = None
        self.left_hand = None

    def equip(self, slot, item):
        prev = getattr(self, slot)
        setattr(self, slot, item)
        return prev
    
    def unequip(self, slot):
        item = getattr(self, slot)
        setattr(self, slot, None)
        return item
    
    

class Player():
    def __init__(self, name, description, status):
        self.name = name
        self.description = description
        self.status = Status(status)

        self.equipment = Equipment()
        self.inventory = Inventory(30)

    def equip(self, slot, item):
        flag = True
        for key, value in item.restriction.items():
            if self.status.max_status[key] < value:
                flag = False

        if flag == True:
            prev = self.equipment.equip(slot, item)
            if prev is not None:
                self.inventory.add_item(prev)
                for key, value in prev.effect.items():
                    self.status.change_added_value(key, -value)

            for key, value in item.effect.items():
                self.status.change_added_value(key, value)
            
            self.inventory.remove_item(item)

    def unequip(self, slot):
        item = self.equipment.unequip(slot)
        self.inventory.add_item(item)

    def add_item(self, item):
        self.inventory.add_item(item)

    def remove_item(self, item):
        self.inventory.remove_item(item)

    def get_item(self, idx):
        return self.inventory.inventory[idx]

    def print(self):
        print(vars(self))
        print(vars(self.status))
        print(vars(self.equipment))
        print(vars(self.inventory))


In [3]:
data

{'Combat': {'Monster Name': 'Rogue Demon',
  'Monster Description': 'A malevolent creature with burning red eyes and charred wings, cloaked in dark mist. It preys on travelers, disrupting the natural balance of Blackwood Forest.',
  'Status': {'HP': 300,
   'MP': 50,
   'Strength': 40,
   'Skill': 25,
   'Dexterity': 10,
   'Intelligence': 20,
   'Luck': 5,
   'Defense': 20,
   'Speed': 10,
   'Concentration': 15,
   'Reaction': 10,
   'MP Regeneration': 1,
   'HP Regeneration': 1},
  'Reward': {'gold': 200, 'exp': 150},
  'Item': {'Item Type': 'Weapon',
   'Item Name': 'Demon Slasher',
   'Item Description': 'A cursed sword forged in the Underworld. It is exceptionally efficient against demonic beings.',
   'Use Restriction': {'Strength': 25},
   'Effect': {'Strength': 25, 'Luck': -5}}}}

In [4]:
from Enemy import Enemy

def combat(json):
    return Enemy(json)

e = combat(data['Combat'])


In [5]:
print(vars(e))
print(vars(e.status))
print(vars(e.item))

{'name': 'Rogue Demon', 'description': 'A malevolent creature with burning red eyes and charred wings, cloaked in dark mist. It preys on travelers, disrupting the natural balance of Blackwood Forest.', 'reward': {'gold': 200, 'exp': 150}, 'status': <Status.Status object at 0x7f9720d7a6b0>, 'item': <Item.Item object at 0x7f9720d795a0>}
{'status': {'HP': 300, 'MP': 50, 'Strength': 40, 'Skill': 25, 'Dexterity': 10, 'Intelligence': 20, 'Luck': 5, 'Defense': 20, 'Speed': 10, 'Concentration': 15, 'Reaction': 10, 'MP Regeneration': 1, 'HP Regeneration': 1}, 'max_status': {'HP': 300, 'MP': 50, 'Strength': 40, 'Skill': 25, 'Dexterity': 10, 'Intelligence': 20, 'Luck': 5, 'Defense': 20, 'Speed': 10, 'Concentration': 15, 'Reaction': 10, 'MP Regeneration': 1, 'HP Regeneration': 1}, 'added_status': {'HP': 0, 'MP': 0, 'Strength': 0, 'Skill': 0, 'Dexterity': 0, 'Intelligence': 0, 'Luck': 0, 'Defense': 0, 'Speed': 0, 'Concentration': 0, 'Reaction': 0, 'MP Regeneration': 0, 'HP Regeneration': 0}, 'durab

In [6]:
for key, value in e.item.effect.items():
    print(key, value)

Strength 25
Luck -5


In [14]:
status = {'HP': 100, 'MP': 50, 'Strength': 25, 'Skill': 10, 'Dexterity': 10, 'Intelligence': 10, 'Luck': 10, 'Defense': 10, 'Speed': 10, 'Concentration': 10, 'Reaction': 10, 'MP Regeneration': 1, 'HP Regeneration': 1}
player = Player('ABC', 'test player', status)
item = Item(data['Combat']['Item'])

In [15]:
player.print()

{'name': 'ABC', 'description': 'test player', 'status': <Status.Status object at 0x7f9720a422c0>, 'equipment': <__main__.Equipment object at 0x7f9720a54670>, 'inventory': <__main__.Inventory object at 0x7f9720a54160>}
{'status': {'HP': 100, 'MP': 50, 'Strength': 25, 'Skill': 10, 'Dexterity': 10, 'Intelligence': 10, 'Luck': 10, 'Defense': 10, 'Speed': 10, 'Concentration': 10, 'Reaction': 10, 'MP Regeneration': 1, 'HP Regeneration': 1}, 'max_status': {'HP': 100, 'MP': 50, 'Strength': 25, 'Skill': 10, 'Dexterity': 10, 'Intelligence': 10, 'Luck': 10, 'Defense': 10, 'Speed': 10, 'Concentration': 10, 'Reaction': 10, 'MP Regeneration': 1, 'HP Regeneration': 1}, 'added_status': {'HP': 0, 'MP': 0, 'Strength': 0, 'Skill': 0, 'Dexterity': 0, 'Intelligence': 0, 'Luck': 0, 'Defense': 0, 'Speed': 0, 'Concentration': 0, 'Reaction': 0, 'MP Regeneration': 0, 'HP Regeneration': 0}, 'durable': []}
{'helmet': None, 'armor': None, 'pants': None, 'gloves': None, 'shoes': None, 'ring1': None, 'ring2': None, 

In [16]:
vars(item)

{'name': 'Demon Slasher',
 'type': 'Weapon',
 'description': 'A cursed sword forged in the Underworld. It is exceptionally efficient against demonic beings.',
 'restriction': {'Strength': 25},
 'effect': {'Strength': 25, 'Luck': -5}}

In [17]:
player.add_item(item)

In [18]:
player.equip('right_hand', item)

In [19]:
player.print()

{'name': 'ABC', 'description': 'test player', 'status': <Status.Status object at 0x7f9720a422c0>, 'equipment': <__main__.Equipment object at 0x7f9720a54670>, 'inventory': <__main__.Inventory object at 0x7f9720a54160>}
{'status': {'HP': 100, 'MP': 50, 'Strength': 50, 'Skill': 10, 'Dexterity': 10, 'Intelligence': 10, 'Luck': 5, 'Defense': 10, 'Speed': 10, 'Concentration': 10, 'Reaction': 10, 'MP Regeneration': 1, 'HP Regeneration': 1}, 'max_status': {'HP': 100, 'MP': 50, 'Strength': 50, 'Skill': 10, 'Dexterity': 10, 'Intelligence': 10, 'Luck': 5, 'Defense': 10, 'Speed': 10, 'Concentration': 10, 'Reaction': 10, 'MP Regeneration': 1, 'HP Regeneration': 1}, 'added_status': {'HP': 0, 'MP': 0, 'Strength': 25, 'Skill': 0, 'Dexterity': 0, 'Intelligence': 0, 'Luck': -5, 'Defense': 0, 'Speed': 0, 'Concentration': 0, 'Reaction': 0, 'MP Regeneration': 0, 'HP Regeneration': 0}, 'durable': []}
{'helmet': None, 'armor': None, 'pants': None, 'gloves': None, 'shoes': None, 'ring1': None, 'ring2': None, 

In [20]:
player.unequip('right_hand')

In [22]:
player.print()

{'name': 'ABC', 'description': 'test player', 'status': <Status.Status object at 0x7f9720a422c0>, 'equipment': <__main__.Equipment object at 0x7f9720a54670>, 'inventory': <__main__.Inventory object at 0x7f9720a54160>}
{'status': {'HP': 100, 'MP': 50, 'Strength': 50, 'Skill': 10, 'Dexterity': 10, 'Intelligence': 10, 'Luck': 5, 'Defense': 10, 'Speed': 10, 'Concentration': 10, 'Reaction': 10, 'MP Regeneration': 1, 'HP Regeneration': 1}, 'max_status': {'HP': 100, 'MP': 50, 'Strength': 50, 'Skill': 10, 'Dexterity': 10, 'Intelligence': 10, 'Luck': 5, 'Defense': 10, 'Speed': 10, 'Concentration': 10, 'Reaction': 10, 'MP Regeneration': 1, 'HP Regeneration': 1}, 'added_status': {'HP': 0, 'MP': 0, 'Strength': 25, 'Skill': 0, 'Dexterity': 0, 'Intelligence': 0, 'Luck': -5, 'Defense': 0, 'Speed': 0, 'Concentration': 0, 'Reaction': 0, 'MP Regeneration': 0, 'HP Regeneration': 0}, 'durable': []}
{'helmet': None, 'armor': None, 'pants': None, 'gloves': None, 'shoes': None, 'ring1': None, 'ring2': None, 